# Brute-force exhaustive search

In [14]:
from itertools import combinations
from collections import Counter

def find_unique(list):
    unique_list = []
    for x in list:
        if x not in unique_list:
            unique_list.append(x)
    return unique_list 
    
def find_gpu_subsets(gpu_list, target_memory):
    upper_factor = 2
    def is_power_of_two(x):
        return x > 0 and (x & (x - 1)) == 0
 

    def get_subsets(gpu_list):
        subsets = []
        for r in range(1, len(gpu_list) + 1):
            for combination in combinations(gpu_list, r):
                subsets.append(combination)
        return subsets
    

    # Expand GPU list to represent all possible allocations
    expanded_list = []
    idx_2_node = {}
    for node_i, (count, memory) in enumerate(gpu_list):
        for i in range(count):
            idx_2_node[len(expanded_list) + i] = node_i
        expanded_list.extend([memory] * count)
        
    expanded_list_idx = list(range(len(expanded_list)))
    
    subsets = get_subsets(expanded_list_idx)
    valid_subsets = []

    for subset_idx in subsets:
        subset = [expanded_list[i] for i in subset_idx]
        total_memory = sum(subset)
        if total_memory >= target_memory and is_power_of_two(len(subset)) and total_memory < upper_factor*target_memory:
            valid_subsets.append(dict(Counter([idx_2_node[i] for i in subset_idx])))
    
    return find_unique(valid_subsets)

# Example usage
# gpu_list = [(8, 48), (8, 48), (8, 48)]  # 8*GPU1 with 48Gb memory, 8*GPU2 with 48Gb memory, 8*GPU3 with 48Gb memory
# target_memory = 750  # Replace with your desired M value
# result = find_gpu_subsets(gpu_list, target_memory)
# print("Valid subsets:", result)


# DP exhaustive search

In [ ]:
from math import log2
from typing import List, Dict, Any, Tuple
from itertools import combinations
from collections import Counter

def find_unique(list):
    unique_list = []
    for x in list:
        if x not in unique_list:
            unique_list.append(x)
    return unique_list 

def find_gpu_subsets_optimized(gpu_list: List[Tuple], target_memory: int) -> List[Dict[int, int]]:
    """_summary_

    Args:
        gpu_list (List[Tuple]): a list of tuples, each tuple contains the number of GPUs and the memory of each node
                                e.g. [node1, node2, ...] = [(# of GPU1, memory of GPU1), (# of GPU2, memory of GPU2), ...]
        target_memory (int): memory needed for the task

    Returns:
        List[Dict[int, int]]: a list of dictionaries, each dictionary contains the number of GPUs for each node
                                e.g. [{node1: # of GPU1, node2: # of GPU2, ...}, ...]
    """
    
    upper_factor = 1.5

    def is_power_of_two(x):
        return x > 0 and (x & (x - 1)) == 0

    def generate_subsets():
        subsets_tmp = []
        for node_i, (count, memory) in enumerate(gpu_list):
            subsets_tmp.append((node_i, count, memory))
        return subsets_tmp

    def backtrack(index, current_memory, current_count, subset):
        if current_memory >= target_memory and is_power_of_two(current_count):
            valid_subsets.append(dict(subset))
            

        if index == len(subsets) or current_memory >= target_memory * upper_factor:
            return

        node_i, count, memory = subsets[index]
        for use_count in range(count + 1):
            if current_memory + use_count * memory >= target_memory * upper_factor:
                break

            subset[node_i] = subset.get(node_i, 0) + use_count
            backtrack(
                index + 1,
                current_memory + use_count * memory,
                current_count + use_count,
                subset,
            )
            if use_count > 0:
                subset[node_i] -= use_count
                if subset[node_i] == 0:
                    del subset[node_i]

    subsets = generate_subsets()
    valid_subsets = []
    backtrack(0, 0, 0, {})
    
    # iterate over all subsets, if number of gpu for a node is zero, remove it from the subset: {0: 0, 1: 8, 2: 8} -> {1: 8, 2: 8}
    for subset in valid_subsets:
        for node in list(subset.keys()):
            if subset[node] == 0:
                del subset[node]       
    return find_unique(valid_subsets)


# Example usage
gpu_list = [(8, 48), (8, 48), (20, 16)]  # 8*GPU1 with 48Gb memory, 8*GPU2 with 48Gb memory, 8*GPU3 with 48Gb memory
target_memory = 768  # Replace with your desired M value
result2 = find_gpu_subsets_optimized(gpu_list, target_memory)
print("Valid subsets:", result2)

Valid subsets: [{0: 4, 1: 8, 2: 20}, {0: 5, 1: 7, 2: 20}, {0: 5, 1: 8, 2: 19}, {0: 6, 1: 6, 2: 20}, {0: 6, 1: 7, 2: 19}, {0: 6, 1: 8, 2: 18}, {0: 7, 1: 5, 2: 20}, {0: 7, 1: 6, 2: 19}, {0: 7, 1: 7, 2: 18}, {0: 7, 1: 8, 2: 17}, {0: 8, 1: 4, 2: 20}, {0: 8, 1: 5, 2: 19}, {0: 8, 1: 6, 2: 18}, {0: 8, 1: 7, 2: 17}, {0: 8, 1: 8}, {0: 8, 1: 8, 2: 16}]


In [16]:
def is_matched(res1, res2):
    
    if res1 == res2:
        # print("Matched")
        return True
    
    for i in res2:
        if i not in res1:
            # print(f"Not Matched {i} of res2")
            return False

    for i in res1:
        if i not in res2:
            # print(f"Not Matched {i} of res1")
            return False

    if len(res1) != len(res2):
        # print("Not Matched Length")
        return False
    # print("Matched")
    return True

In [ ]:
# random test cases for matching the results
# generate random gpu_list with len(gpu_list) 1, 2, 3, 4
# generate random target_memory
# check if the results are matched
import random

sample_num = 200
track_matched = 0
for i in range(sample_num):
    gpu_list = [(random.randint(1, 8), random.randint(1, 100)) for i in range(random.randint(1, 3))]
    target_memory = random.randint(1, 1000)
    res1 = find_gpu_subsets(gpu_list, target_memory)
    res2 = find_gpu_subsets_optimized(gpu_list, target_memory)
    matched = is_matched(res1, res2)
    if matched:
        track_matched += 1
    if not matched:
        print(f"gpu_list: {gpu_list}, target_memory: {target_memory}, res1: {res1}, res2: {res2}")
    
print(f"Matched: {track_matched}/{sample_num}")

Matched: 0/200


In [21]:
gpu_list= [(5, 68), (5, 29), (3, 85)]
target_memory= 77
#, res1: [{2: 1}, {0: 2}, {0: 1, 1: 1}, {0: 1, 2: 1}, {1: 1, 2: 1}, {1: 4}], res2: [{2: 1}, {1: 1, 2: 1}, {1: 4}, {1: 4}, {0: 1, 2: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 2}, {0: 2}, {0: 2}]

res1 = find_gpu_subsets(gpu_list, target_memory)
res2 = find_gpu_subsets_optimized(gpu_list, target_memory)
is_matched(res1, res2)
(target_memory, res1, res2)

(77,
 [{2: 1}, {0: 2}, {0: 1, 1: 1}, {0: 1, 2: 1}, {1: 1, 2: 1}, {1: 4}],
 [{2: 1}, {1: 1, 2: 1}, {1: 4}, {0: 1, 2: 1}, {0: 1, 1: 1}, {0: 2}])